In [ ]:
#Download Galaxy zoo tables and SDSS spec objects and cross-match
from astroquery.sdss import SDSS
from astropy import coordinates as coords
from astropy.io import fits
import numpy as np
from PIL import Image
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from astropy.table import Table,vstack,Column,unique
import copy
import os.path

In [ ]:
xid = SDSS.query_sql('select count(*) from zooSPec')
print('total galaxies =',xid.field('Column1')) #hay 667944 galaxias
ngal=xid
xid = SDSS.query_sql('select objid,p_el,p_cw,p_acw,p_edge,p_dk,p_mg,p_cs from zooSpec where p_el < 0.5 and objid>0')
ngal1=len(xid)
xid2 = SDSS.query_sql('select objid,p_el,p_cw,p_acw,p_edge,p_dk,p_mg,p_cs from zooSpec where p_el >= 0.5 and objid>0')
ngal2=len(xid2)
print('ngal1 ngal2 ntot',ngal1,ngal2,ngal1+ngal2)
xid3=vstack([xid,xid2])
xid3.write('zoospecnew.data',format='ascii')
print('xid3 nrows=',len(xid3))

In [ ]:
xid = Table.read('zoospecnew.data',format='ascii')
xid.sort(['objid'])

In [ ]:
ngal=len(xid)
nbuf=300
niter=int(ngal/nbuf)-1
print('niter=%d tot=%d'%(niter,niter*nbuf))
for i in range(niter):
    objid=xid['objid'][i*nbuf:((i+1)*nbuf)]
    p_el=xid['p_el'][i*nbuf:((i+1)*nbuf)]
    p_cw=xid['p_cw'][i*nbuf:((i+1)*nbuf)]
    p_acw=xid['p_acw'][i*nbuf:((i+1)*nbuf)]
    p_edge=xid['p_edge'][i*nbuf:((i+1)*nbuf)]
    p_dk=xid['p_dk'][i*nbuf:((i+1)*nbuf)]
    p_mg=xid['p_mg'][i*nbuf:((i+1)*nbuf)]
    p_cs=xid['p_cs'][i*nbuf:((i+1)*nbuf)]
    #print(len(objid))
    for k in range(nbuf):
        if k==0:
            stobj='''%s'''%objid[0]
        if k>0:
            stobj=stobj + ',''%s'''%objid[k]
    xid2 = SDSS.query_sql('select objid,run,rerun,camcol,field,rowc,colc,petrorad_r,rowc_g,rowc_i,colc_g,colc_i from Galaxy where objid in (%s) order by objid asc'%(stobj))
    nres=len(xid2)
    
    c1=Column(name='p_el',data=np.zeros(nres,dtype=np.float32))
    c2=Column(name='p_cw',data=np.zeros(nres,dtype=np.float32))
    c3=Column(name='p_acw',data=np.zeros(nres,dtype=np.float32))
    c4=Column(name='p_edge',data=np.zeros(nres,dtype=np.float32))
    c5=Column(name='p_dk',data=np.zeros(nres,dtype=np.float32))
    c6=Column(name='p_mg',data=np.zeros(nres,dtype=np.float32))
    c7=Column(name='p_cs',data=np.zeros(nres,dtype=np.float32))                
    xid2.add_columns([c1,c2,c3,c4,c5,c6,c7])
    
    #if nres<nbuf:
    #    print('no galactic elements i=%d nres=%d nbuf=%d\n'%(i,nres,nbuf))

    i1=0
    i2=0
    for k in range(nres):
        objref=xid2['objid'][i2]
        oid=objid[i1]
        if (oid==objref):
            xid2['p_el'][i2]=p_el[i1]
            xid2['p_cw'][i2]=p_cw[i1]
            xid2['p_acw'][i2]=p_acw[i1]
            xid2['p_edge'][i2]=p_edge[i1]
            xid2['p_dk'][i2]=p_dk[i1]
            xid2['p_mg'][i2]=p_mg[i1]
            xid2['p_cs'][i2]=p_cs[i1]
            i1=i1+1
            i2=i2+1
        if (oid!=objref):
            #print('prev line %d %d %d\n'%(i2-1,objid[i1-1],xid2['objid'][i2-1]))
            #print('No galaxy in i=%d id=%d p_el=%f'%(i2,objid[i1],p_el[i1]))
            #print('next line %d %d\n'%(i2+1,objid[i1+1]))
            i1=i1+1
    #print(xid2)
            
    if i==0:
        xid3=copy.deepcopy(xid2)
    if i>0:
        xid4=vstack([xid3,xid2])
        xid3=copy.deepcopy(xid4)
    if i%50==0:
        print('i=%d niter=%d len(xid3)=%d ngal=%d'%(i,niter,len(xid3),ngal))


print(len(xid3))
xid3.write('zoospecnewall.data',format='ascii',overwrite=True)

In [ ]:
xid3.show_in_notebook()

In [ ]:
#READ FULL CATALOG - CREATE IMAGE LIST TO DOWNLOAD
xid = Table.read('zoospecnewall.data',format='ascii')
ngal=len(xid)
col=Column(name='imagename',data=["                  " for x in range(ngal)])                
xid.add_column(col)
for i in range(ngal):
    v1=xid['run'][i]
    v2=xid['rerun'][i]
    v3=xid['camcol'][i]
    v4=xid['field'][i]
    ss='%d_%d_%d_%d'%(v1,v2,v3,v4)
    xid['imagename'][i] =ss

xuniq=unique(xid,keys=['imagename'])
nuniq=len(xuniq)
print('uniq images =%d'%nuniq)
xid.write('zoospecnewall2.data',format='ascii')
xuniq.write('unique.data',format='ascii')